In [1]:
import pandas as pd
import numpy as np
import datetime
import os

ModuleNotFoundError: No module named 'pandas'

# Daily Data by Health Board

The following Github Repo https://github.com/DataScienceScotland/COVID-19-Management-Information contains information regarding scottish information which is pulled on a daily basis. 

In [2]:
df_cases = pd.read_csv("https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20Cumulative%20cases.csv")
df_hospital_confirmed = pd.read_csv("https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20Hospital%20patients%20-%20Confirmed.csv")
df_hospital_all = pd.read_csv("https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20Hospital%20patients.csv")
df_icu = pd.read_csv("https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20ICU%20patients.csv")

In [3]:
df_icu.head()

,Date,Ayrshire and Arran,Borders,Dumfries and Galloway,Fife,Forth Valley,Grampian,Greater Glasgow and Clyde,Highland,Lanarkshire,Lothian,Orkney,Shetland,Tayside,Western Isles,The Golden Jubilee National Hospital
0,2020-03-18,*,*,*,*,*,*,*,*,*,5,*,*,*,*,*
1,2020-03-19,*,*,*,*,*,*,*,*,*,5,*,*,*,*,*
2,2020-03-20,*,*,*,*,*,*,*,*,*,6,*,*,*,*,*
3,2020-03-21,*,*,*,*,*,*,*,*,*,9,*,*,*,*,*
4,2020-03-22,*,*,*,*,*,*,*,*,*,7,*,*,*,*,*


In [47]:
hbo_list = list(df_icu.columns.values)
hbo_list

['Date',
 'Ayrshire and Arran',
 'Borders',
 'Dumfries and Galloway',
 'Fife',
 'Forth Valley',
 'Grampian',
 'Greater Glasgow and Clyde',
 'Highland',
 'Lanarkshire',
 'Lothian',
 'Orkney',
 'Shetland',
 'Tayside',
 'Western Isles',
 'The Golden Jubilee National Hospital']

In [5]:
path_dict = {
                "cases" : "https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20Cumulative%20cases.csv",
                "hospital_covid" : "https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20Hospital%20patients%20-%20Confirmed.csv",
                "hospital_all" : "https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20Hospital%20patients.csv",
                "icu" : "https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20ICU%20patients.csv",
            }

In [48]:
hbo_dict = {
    
    1 : 'Ayrshire and Arran',
    2 : 'Borders',
    3 : 'Dumfries and Galloway',
    4 : 'Western Isles',
    5 : 'Fife',
    6 : 'Forth Valley',
    7 : 'Grampian',
    8 : 'Greater Glasgow and Clyde',
    9 : 'Highland',
    10 : 'Lanarkshire',
    11 : 'Lothian',
    12 : 'Orkney',
    13 : 'Shetland',
    14 : 'Tayside'
}

In [54]:
hbo_dict[14]

'Tayside'

In [6]:
def convert_date_to_week(date):
    date_list = date.split('-')
    date_num = list(map(int, date_list))
    a_date = datetime.date(date_num[0], date_num[1], date_num[2])
    week_number = a_date.isocalendar()[1]
    return week_number

In [7]:
def convert_date_to_day(date):
    date_list = date.split('-')
    date_num = list(map(int, date_list))
    a_date = datetime.date(date_num[0], date_num[1], date_num[2])
    day_of_year = (a_date - datetime.date(date_num[0], 1, 1)).days + 1
    return day_of_year

In [8]:
a = "2020-03-21"
convert_date_to_day(a)

81

In [9]:
def get_hbo_data_by_date(health_board, data_type):
    df = pd.read_csv(path_dict.get(data_type))
    df = df[['Date', health_board]]
    df = df.rename(columns={'Date':'date', health_board:'value'})
    df = df.iloc[-31:]
    return df

In [10]:
def get_hbo_data_by_days(health_board, data_type):
    df = pd.read_csv(path_dict.get(data_type))
    df = df[['Date', health_board]]
    df['day_of_year'] = df['Date'].apply(lambda x : convert_date_to_day(x))
    df = df.set_index('day_of_year')
    df = df.drop(columns=['Date'])
    df = df.reset_index(inplace=False)
    return df

In [11]:
def get_last_month_data(df):
    df = df.iloc[-31:]
    return df

In [61]:
def get_all_hbo_data(hbo_list, path_dict, root_path):
    
    for i in range (1, len(hbo_dict)+1):
        for data_type in path_dict.keys():
            
            # Create the dataframe
            print(hbo_dict[i])
            df = get_hbo_data_by_date(hbo_dict[i], data_type)
            path = root_path + f"{i}_{data_type}.csv"
            current_dir = os.getcwd()
            curr = os.path.dirname(os.getcwd())
            #print(curr)
            df.to_csv(current_dir + path)

In [62]:
get_all_hbo_data(hbo_list, path_dict, "/data/hbo/")

Ayrshire and Arran
Ayrshire and Arran
Ayrshire and Arran
Ayrshire and Arran
Borders
Borders
Borders
Borders
Dumfries and Galloway
Dumfries and Galloway
Dumfries and Galloway
Dumfries and Galloway
Western Isles
Western Isles
Western Isles
Western Isles
Fife
Fife
Fife
Fife
Forth Valley
Forth Valley
Forth Valley
Forth Valley
Grampian
Grampian
Grampian
Grampian
Greater Glasgow and Clyde
Greater Glasgow and Clyde
Greater Glasgow and Clyde
Greater Glasgow and Clyde
Highland
Highland
Highland
Highland
Lanarkshire
Lanarkshire
Lanarkshire
Lanarkshire
Lothian
Lothian
Lothian
Lothian
Orkney
Orkney
Orkney
Orkney
Shetland
Shetland
Shetland
Shetland
Tayside
Tayside
Tayside
Tayside


In [ ]:
get_all_hbo_data2()

In [121]:
get_hbo_data_by_date('Orkney', "hospital_all")

,date,value
17,2020-04-04,6
18,2020-04-05,*
19,2020-04-06,*
20,2020-04-07,*
21,2020-04-08,*
22,2020-04-09,*
23,2020-04-10,*
24,2020-04-11,*
25,2020-04-12,*
26,2020-04-13,*


In [65]:
df_fife = get_hbo_data_by_date('Fife', 'hospital_covid')

In [67]:
df_fife

,date,value
16,2020-04-11,97
17,2020-04-12,99
18,2020-04-13,101
19,2020-04-14,102
20,2020-04-15,92
21,2020-04-16,96
22,2020-04-17,98
23,2020-04-18,91
24,2020-04-19,105
25,2020-04-20,105


In [43]:
df_fife_small = get_last_month_data(df_fife)

In [44]:
df_fife_small

,day_of_year,Fife
16,102,97
17,103,99
18,104,101
19,105,102
20,106,92
21,107,96
22,108,98
23,109,91
24,110,105
25,111,105


In [17]:
df_fife2 = get_hbo_data2('Fife', 'hospital_covid')
df_fife2 = df_fife2.rename(columns={'Date':'date', 'Fife':'value'})

In [20]:
df_fife2.to_csv('testdata_basic_linegraph.csv')

In [34]:
df_fife = df_fife.rename(columns={'day_of_year':'time', 'Fife':'valueA'})

In [35]:
df_fife.to_csv('fife_hospital_all2.csv')

In [11]:
df_fife

,day_of_year,Fife
0,78,18
1,79,24
2,80,20
3,81,16
4,82,23
5,83,28
6,84,42
7,85,28
8,86,45
9,87,51


In [50]:
df_fife['day_of_year'] = df_fife['Date'].apply(lambda x : convert_date_to_day(x))

In [53]:
df_fife

,Date,Fife,Date_new,day_of_year
0,2020-03-18,18,78,78
1,2020-03-19,24,79,79
2,2020-03-20,20,80,80
3,2020-03-21,16,81,81
4,2020-03-22,23,82,82
5,2020-03-23,28,83,83
6,2020-03-24,42,84,84
7,2020-03-25,28,85,85
8,2020-03-26,45,86,86
9,2020-03-27,51,87,87


In [118]:
df_hospital_all

,Date,Ayrshire and Arran,Borders,Dumfries and Galloway,Fife,Forth Valley,Grampian,Greater Glasgow and Clyde,Highland,Lanarkshire,Lothian,Orkney,Shetland,Tayside,Western Isles,The Golden Jubilee National Hospital
0,2020-03-18,*,*,*,18,*,*,18,*,46,41,*,6,10,*,*
1,2020-03-19,31,11,*,24,5,*,22,16,37,39,*,8,17,*,*
2,2020-03-20,37,7,5,20,6,*,32,11,53,48,*,6,17,*,*
3,2020-03-21,27,12,5,16,8,6,42,14,37,49,*,*,23,*,*
4,2020-03-22,38,18,6,23,8,7,57,28,44,24,*,*,26,*,*
5,2020-03-23,25,10,7,28,13,6,70,36,49,53,*,*,26,*,*
6,2020-03-24,36,10,20,42,14,5,87,43,64,88,*,*,30,*,*
7,2020-03-25,41,16,20,28,19,7,104,39,80,86,*,*,40,*,*
8,2020-03-26,45,9,23,45,62,8,125,39,85,79,*,*,50,*,*
9,2020-03-27,44,25,31,51,64,13,153,28,92,115,*,*,50,*,*
